Skeleton in pybullet: physcap.urdf

Vnet result (on sequence): motion.motion, ddd.mdd

In [1]:
import numpy as np

data_path = 'data/sample_vnect_2ds.npy'

data = np.load(data_path) #(14235, 17, 2)

print(data.shape)

(14235, 17, 2)


In [2]:
data_path = 'data/vnect_motions.npy'

data = np.load(data_path) #(33, 600) #dimension related to dofs

print(data.shape)

(33, 600)


In [3]:
def dof_name_getter(skeleton_filename):
    with open(skeleton_filename) as f:
        content = f.readlines()
    content = np.array([x.strip().split(" ") for x in content], dtype=object)
    dof_names_mess = np.array(content[99:][np.arange(0, len(content[99:]), 3)[:-1]])
    dof_names = [x[0] for x in dof_names_mess]
    return dof_names

skeleton_filename = 'asset/physcap.skeleton'

dof_names = dof_name_getter(skeleton_filename)

print(len(dof_names))
print(dof_names)

33
['trans_root_tx', 'trans_root_ty', 'trans_root_tz', 'rot_root_rx', 'rot_root_ry', 'rot_root_rz', 'rot_left_hip_rx', 'rot_left_hip_ry', 'rot_left_hip_rz', 'rot_right_hip_rx', 'rot_right_hip_ry', 'rot_right_hip_rz', 'rot_left_knee_rx', 'rot_right_knee_rx', 'rot_left_ankle_rx', 'rot_left_ankle_ry', 'rot_right_ankle_rx', 'rot_right_ankle_ry', 'rot_neck_rx', 'rot_neck_ry', 'rot_neck_rz', 'rot_left_clavicle_ry', 'rot_left_clavicle_rz', 'rot_right_clavicle_ry', 'rot_right_clavicle_rz', 'rot_left_shoulder_ry', 'rot_left_shoulder_rz', 'rot_right_shoulder_ry', 'rot_right_shoulder_rz', 'rot_left_elbow_rx', 'rot_left_elbow_ry', 'rot_right_elbow_rx', 'rot_right_elbow_ry']


In [4]:
# data_path = 'results/PhyCap_q.npy' #(4792, 43)
# data_path = 'results/VNect_q.npy' #(2408, 43)
data_path = 'results/pred_con.npy' #(14235, 1, 4)
# data_path = 'results/pred_sta.npy' #(14235, 1)

data = np.load(data_path) #(33, 600) #dimension related to dofs

print(data.shape)
print(data[0])

(14235, 1, 4)
[[1. 1. 1. 1.]]


In [20]:
import joblib
import os
data_dir = '/home/datassd/yuxuan/data_event_out'
action = 'subject01_group1_time1'
num_frame = len(os.listdir('%s/pose_events/%s' % (data_dir, action))) - 1
theta_list, tran_list, joints2d_list, joints3d_list = [], [], [], []
for idx in range(num_frame):
    beta, theta, tran, joints3d, joints2d = joblib.load(
                '%s/pose_events/%s/pose%04i.pkl' % (data_dir, action, idx))
    theta_list.append(theta[0])
    tran_list.append(tran[0])
    joints2d_list.append(joints2d)
    joints3d_list.append(joints3d)
    # print(tran.shape)
    # print(tran[0])

[-0.04620038  0.21895348  3.53260262]
[-0.04708561  0.21840105  3.53692778]
[-0.04934958  0.21854843  3.53702747]
[-0.04965324  0.22051129  3.53917158]
[-0.05062289  0.21745876  3.53695955]
[-0.05051085  0.21958378  3.53812876]
[-0.0495018   0.2216079   3.53918507]
[-0.04931634  0.21978342  3.54042791]
[-0.04893226  0.22349641  3.54437826]
[-0.05088661  0.23974749  3.54867071]
[-0.04852368  0.27445603  3.56008358]
[-0.05118573  0.30229438  3.56707132]
[-0.05190164  0.35097576  3.57778383]
[-0.05405071  0.39717301  3.58836935]
[-0.05962789  0.43756862  3.60498902]
[-0.05787728  0.47553194  3.61177238]
[-0.05894075  0.50397813  3.61319774]
[-0.0567997   0.51348473  3.60864344]
[-0.05966042  0.49356308  3.59107593]
[-0.05706832  0.47190492  3.584133  ]
[-0.05729297  0.38366148  3.55890952]
[-0.0548324   0.28756419  3.53600678]
[-0.05841843  0.16699078  3.51599299]
[-0.05544563  0.06417236  3.49621107]
[-0.05261575  0.03859765  3.48499226]
[-0.05025506  0.04994127  3.48188601]
[-0.0496565 

In [15]:
kinematic_2d = np.stack(joints2d_list, axis=0)
print(kinematic_2d.shape)
print(kinematic_2d[0,0])

(1345, 24, 3)
[123.61658543 126.52548169   3.55519741]


In [7]:
data_path = 'results/contacts.npy' #(1345, 1, 4)
# data_path = 'results/stationary.npy' #(1345, 1)

data = np.load(data_path) #(33, 600) #dimension related to dofs

print(data.shape)
print(data)

(1345, 1, 4)
[[[1. 1. 0. 0.]]

 [[1. 1. 0. 0.]]

 [[1. 1. 0. 0.]]

 ...

 [[1. 1. 0. 0.]]

 [[1. 0. 0. 0.]]

 [[0. 0. 0. 0.]]]


In [13]:
import torch
theta = torch.zeros((1,3))
l1norm = torch.norm(theta + 1e-8, p=2, dim=1)
angle = torch.unsqueeze(l1norm, -1)
normalized = torch.div(theta, angle)
angle = angle * 0.5
v_cos = torch.cos(angle)
v_sin = torch.sin(angle)
quat = torch.cat([v_cos, v_sin * normalized], dim=1)
print(l1norm)

tensor([1.7321e-08])


In [2]:
import pickle
kinect_extr = pickle.load(open('/home/datassd/yuxuan/data_event_out/calib_1024/kinect_extrinsic_param.pkl', 'rb'))
print('kinect_extr', kinect_extr.keys())

extr = pickle.load(open('/home/datassd/yuxuan/data_event_out/calib_1024/extrinsic_param_1024.pkl', 'rb'))
print('extr', extr.keys())


intr = pickle.load(open('/home/datassd/yuxuan/data_event_out/calib_1024/intrinsic_param.pkl', 'rb'))
print('intr', intr.keys())


kinect_extr dict_keys(['azure_kinect_0_d2c', 'azure_kinect_1_d2c', 'azure_kinect_2_d2c', 'kinect_v2_1_d2c', 'kinect_v2_2_d2c'])
extr dict_keys(['azure_kinect_2-kinect_v2_2', 'azure_kinect_0-azure_kinect_2', 'polar-azure_kinect_0', 'event_camera-azure_kinect_0', 'azure_kinect_0-azure_kinect_1', 'azure_kinect_1-kinect_v2_1'])
intr dict_keys(['azure_kinect_0_color', 'azure_kinect_0_depth', 'azure_kinect_1_color', 'azure_kinect_1_depth', 'azure_kinect_2_color', 'azure_kinect_2_depth', 'kinect_v2_1_color', 'kinect_v2_1_depth', 'kinect_v2_2_color', 'kinect_v2_2_depth', 'event_camera', 'polar'])


In [4]:
print(kinect_extr['azure_kinect_0_d2c'])
print('depth', intr['azure_kinect_0_depth'])
print('event_camera', intr['event_camera'])

from scipy.spatial.transform import Rotation as Rot  

r2 = Rot.from_matrix(kinect_extr['azure_kinect_0_d2c'][0])
print(r2.as_euler('zyx', degrees=True))


(array([[ 9.99998858e-01,  8.06783638e-04, -1.27755243e-03],
       [-6.74956529e-04,  9.94985107e-01,  1.00020901e-01],
       [ 1.35184087e-03, -1.00019925e-01,  9.94984516e-01]]), array([[-32.20522772],
       [ -2.42119739],
       [  0.17579252]]))
(503.0663909874723, 503.2110805572531, 311.33891831250764, 321.9362422108562, 576, 640, -28.34559721362226, 405.10560089798076, -0.00018551407336784846, 0.001373663043499764, 75.66967336693958, -27.991398216341494, 395.1877482416642, 215.6202849517827)
(976.7838346046938, 976.6954700661433, 1000.7015978283123, 784.6995695911885, 1536, 2048, 13.366022046162477, -11.15344330351859, 0.00016147564080253588, -0.008163149588165207, -88.58085389918612, 13.318338772065024, -12.813268592939618, -83.7145525534381)
[-0.04622534 -0.07319838 -5.74037876]
